In [1]:
import os
import json
import uuid
from datetime import datetime, timezone
from neo4j import GraphDatabase
from dotenv import load_dotenv
from google import genai

load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("GOOGLE_API_KEY not found in environment variables. Please set it in your .env file.")

client = genai.Client(api_key=GOOGLE_API_KEY)

search_products_tool = {
    "name": "search_products",
    "description": "Searches the product catalog for items based on a text query and optional filters.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "A search query for products."
            },
            "filters": {
                "type": "object",
                "description": "Optional filters like color.",
                "properties": {
                    "color": {"type": "string"}
                }
            }
        },
        "required": ["query"]
    }
}

search_faqs = {
    "name": "search_faqs",
    "description": "Searches a knowledge base of Frequently Asked Questions (FAQs) based on a user's query. Returns a list of relevant FAQs including their questions, answers, and categories.",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The user's question or search query to find relevant FAQs. For example: 'How long does shipping take?' or 'return policy'."
            }
        },
        "required": ["text"]
    }
}

In [5]:
import os
from google import genai
from google.genai import types

# Example Functions
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_weather_forecast(location={location})")
    # TODO: Make API call
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 25, "unit": "celsius"}


def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    # TODO: Interact with a thermostat API
    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

client = genai.Client()
config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature])

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    config=config,)

# Print the final, user-facing response
print(response.text)

Tool Call: get_weather_forecast(location=London)
Tool Response: {'temperature': 25, 'unit': 'celsius'}
Tool Call: set_thermostat_temperature(temperature=20)
Tool Response: {'status': 'success'}
It is 25°C in London, so I have set the thermostat to 20°C.


In [6]:
turn_on_the_lights_schema = {'name': 'turn_on_the_lights'}
turn_off_the_lights_schema = {'name': 'turn_off_the_lights'}

prompt = """
  Hey, can you write run some python code to turn on the lights, wait 10s and then turn off the lights?
  """

tools = [
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]}
]

await run(prompt, tools=tools, modality="AUDIO")

NameError: name 'run' is not defined

In [13]:
prompt = """
  List a few popular cookie recipes
"""

In [14]:
MODEL_ID = "gemini-2.5-flash"

raw_response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config={
        'response_mime_type': 'application/json'
    },
)

In [15]:
import json

response = json.loads(raw_response.text)
print(response)

[{'name': 'Chocolate Chip Cookies', 'description': 'A classic American cookie, known for its buttery, chewy texture and melty chocolate chips. Often made with a mix of brown and white sugar for depth of flavor.', 'popularity_rank': 1}, {'name': 'Oatmeal Raisin Cookies', 'description': 'Chewy cookies made with rolled oats and raisins, often spiced with cinnamon. A comforting and hearty option.', 'popularity_rank': 2}, {'name': 'Peanut Butter Cookies', 'description': 'Rich and dense cookies with a distinct peanut butter flavor, traditionally flattened with a fork to create a criss-cross pattern.', 'popularity_rank': 3}, {'name': 'Sugar Cookies', 'description': 'Simple, buttery cookies that serve as a perfect base for decorating with icing and sprinkles. Popular around holidays.', 'popularity_rank': 4}, {'name': 'Snickerdoodles', 'description': 'Soft and chewy cookies coated in a mixture of cinnamon and sugar, often characterized by a slight tangy taste from cream of tartar.', 'popularity

In [16]:
print(json.dumps(response, indent=4))

[
    {
        "name": "Chocolate Chip Cookies",
        "description": "A classic American cookie, known for its buttery, chewy texture and melty chocolate chips. Often made with a mix of brown and white sugar for depth of flavor.",
        "popularity_rank": 1
    },
    {
        "name": "Oatmeal Raisin Cookies",
        "description": "Chewy cookies made with rolled oats and raisins, often spiced with cinnamon. A comforting and hearty option.",
        "popularity_rank": 2
    },
    {
        "name": "Peanut Butter Cookies",
        "description": "Rich and dense cookies with a distinct peanut butter flavor, traditionally flattened with a fork to create a criss-cross pattern.",
        "popularity_rank": 3
    },
    {
        "name": "Sugar Cookies",
        "description": "Simple, buttery cookies that serve as a perfect base for decorating with icing and sprinkles. Popular around holidays.",
        "popularity_rank": 4
    },
    {
        "name": "Snickerdoodles",
        "de